In [1]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np
from tpot import TPOTClassifier

# Read in data
z_data = pd.read_csv('fse17_act_deact_zscore_groups.csv',sep=';',decimal='.')

#first, drop everything non-numeric ===========================================================
z_data = z_data.drop('scan', axis = 1)
z_data = z_data.drop('trial', axis = 1)
z_data = z_data.drop('response', axis = 1)
z_data = z_data.drop('snippet', axis = 1)

numLabels = 2 # num of different labels (comprehension and rest)
sizeTrainSet = 11 # num of participants used for training
sizeTestSet = 3 # num participants used for testing
numParticipants = sizeTrainSet + sizeTestSet

C:\Users\author\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\author\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory C:\Users\author\Anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnin

In [2]:
#group the data and get the labels after the grouping
#return the grouped data and the labels
def prepare(data,groupingColumns):
    grouped = data.groupby(groupingColumns)
    groupedAgg = grouped.aggregate(np.mean)
    labels = groupedAgg.index.get_level_values(level='task')
    return groupedAgg,labels

In [3]:
#split according to participants (first 12 and last 4)
#return all four sets (much like train_test_split from sklearn)
#TODO: introduce variation (see file coarseAverageParticipantSplit)

def split(features, labels, low, high):
    
    #=== creating training and validation set ======================================================
    # Saving feature names for later use
    feature_list = list(features.columns)

    # Convert to numpy array
    features = np.array(features)
    labels = np.array(labels)

    #get the rows excluding the last indexed (i.e., row with the index 587 is the last to include)
    training_features = features[0:low]
    training_features = np.array(training_features)

    testing_features = features[low:high]
    testing_features = np.array(testing_features)

    training_target = labels[0:low]
    training_target = np.array(training_target)
    testing_target = labels[low:high]
    testing_target = np.array(testing_target)
    #=== end creating training and validation set ===================================================
    
    return training_features, testing_features, training_target, testing_target

In [4]:
def findIndices(name):
    numGroups = int(name.split('_')[1])
    low = sizeTrainSet*numGroups*numLabels
    high = numParticipants*numGroups*numLabels
    return low, high



In [ ]:
import pdb

#define the columns to group by
groupingColumn1 = 'proband'
groupingColumn2 = 'task'
groupingColumn3 = ''

#define how many different variants of aggregation and aggregation levels
numVariationsPerAggLevel = 10
numAggLevels = 40
numAggregationVariants = numVariationsPerAggLevel * numAggLevels

#get the index of the starting column that contains the labels of the groupings
offset = z_data.columns.get_loc('aggr_2_groups0')
print(offset)
collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):
    # group the data set
#     pdb.set_trace()
    #=============================================================================================================
    columnsForGrouping = [groupingColumn1,groupingColumn2,z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================
    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
    #=============================================================================================================
    print(offset)
    print(low, high)
    
    folder = 'AllROIs/'
    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'
    #=============================================================================================================
    # run t_pot
    # TODO: extract method
    tpot = TPOTClassifier(generations=5, population_size=5, verbosity=2, n_jobs=20,max_eval_time_mins=1)
    tpot.fit(training_features, training_target)
    print(tpot.score(testing_features, testing_target))
    tpot.export(folder + fileName)
#     #=============================================================================================================
    
    offset = offset + 1


In [8]:
#get the index of the starting column that contains the labels of the groupings
offset = z_data.columns.get_loc('aggr_2_groups0')
print(offset)
collectAccuracies = []

#collect all instances where unmatched is not possible.
unmatched = []

#loop through the columns
#for each column, group the data set
# transform it to numpy.ndarray and split into training and validation set
# run t_pot
# and do the learning
while offset < len(z_data.columns):    

    #=============================================================================================================
    columnsForGrouping = ['proband','task',z_data.columns[offset]]
    features, labels = prepare(z_data,columnsForGrouping)
    #=============================================================================================================
    #=============================================================================================================
    # transform to numpy.ndarray and do split
    # find low and high
    low, high = findIndices(z_data.columns[offset])
    training_features, testing_features, training_target, testing_target = split(features, labels, low, high)
    #=============================================================================================================
    
    fileName = 'tpot_mnist_pipeline_triangulateAggregationLevelParticipantSplit' + z_data.columns[offset] + '.py'

    #=============================================================================================================
    #do the learning
    # TODO: extract method
    with open(folder + fileName) as f:
        content = f.readlines()
    cleanedContent = []
    for line in content:
        if 'tpot_data' not in line and 'training_target, testing_target' not in line:
            cleanedContent.append(line)
    
    fileNameCleaned = folder + 'cleaned_' + fileName
    with open(fileNameCleaned, 'w') as filehandle:  
        for line in cleanedContent:
            filehandle.write('%s\n' % line)
            
    %run -i $fileNameCleaned
    
    # print the accuracy
    # TODO: extract method
    # TODO: print all accurracy values to one file
    num_matches = 0;
    for a, b in zip(testing_target, results):
        if a == b:
            num_matches = num_matches + 1
        else:
            unmatched.append((a,b))
    print('Number of matches:',num_matches,'(of',testing_target.size,')')

    accuracy = num_matches/testing_target.size*100
    print('Accuary: ',accuracy)
    #=============================================================================================================
    
    #=============================================================================================================
    #collect all accuracy values to plot them
    collectAccuracies.append(accuracy)
    
    
    offset = offset + 1
    

392
392
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
393
44 56
Number of matches: 7 (of 12 )
Accuary:  58.333333333333336
394
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
395
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
396
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
397
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
398
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
399
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
400
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
401
44 56
Number of matches: 12 (of 12 )
Accuary:  100.0
402
66 84
Number of matches: 17 (of 18 )
Accuary:  94.44444444444444
403
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
404
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
405
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
406
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
407
66 84


C:\Users\author\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\author\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 17 (of 18 )
Accuary:  94.44444444444444
408
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
409
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
410
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
411
66 84
Number of matches: 18 (of 18 )
Accuary:  100.0
412
198 252
Number of matches: 49 (of 54 )
Accuary:  90.74074074074075
413
198 252
Number of matches: 48 (of 54 )
Accuary:  88.88888888888889
414
198 252
Number of matches: 52 (of 54 )
Accuary:  96.29629629629629
415
198 252
Number of matches: 50 (of 54 )
Accuary:  92.5925925925926
416
198 252
Number of matches: 50 (of 54 )
Accuary:  92.5925925925926
417
198 252
Number of matches: 52 (of 54 )
Accuary:  96.29629629629629
418
198 252
Number of matches: 53 (of 54 )
Accuary:  98.14814814814815
419
198 252
Number of matches: 51 (of 54 )
Accuary:  94.44444444444444
420
198 252
Number of matches: 51 (of 54 )
Accuary:  94.44444444444444
421
198 252
Number of matches: 48 (of 54 )
Accuary:  88.88888888888889
422
2

C:\Users\author\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 70 (of 78 )
Accuary:  89.74358974358975
424
286 364
Number of matches: 70 (of 78 )
Accuary:  89.74358974358975
425
286 364
Number of matches: 69 (of 78 )
Accuary:  88.46153846153845
426
286 364
Number of matches: 70 (of 78 )
Accuary:  89.74358974358975
427
286 364
Number of matches: 68 (of 78 )
Accuary:  87.17948717948718
428
286 364
Number of matches: 74 (of 78 )
Accuary:  94.87179487179486
429
286 364


C:\Users\author\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\author\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Number of matches: 71 (of 78 )
Accuary:  91.02564102564102
430
286 364
Number of matches: 73 (of 78 )
Accuary:  93.58974358974359
431
286 364
Number of matches: 73 (of 78 )
Accuary:  93.58974358974359


IndexError: list index out of range

In [7]:
len(z_data.columns)

2230

In [9]:
countComprehensionRest = 0
countComprehensionSyntax = 0
countRestSyntax = 0
for x in unmatched:
    if x == (0,1) or x == (1,0):
        countComprehensionRest = countComprehensionRest + 1
    if x == (0,2) or x == (2,0):
        countComprehensionSyntax = countComprehensionSyntax + 1
    if x == (1,2) or x == (2,1):
        countRestSyntax = countRestSyntax + 1
print('Comprehension/Rest',countComprehensionRest)
print('Comprehension/Syntax',countComprehensionSyntax)
print('Rest/Syntax',countRestSyntax)

Comprehension/Rest 114
Comprehension/Syntax 0
Rest/Syntax 0


In [10]:
def split_list(alist, wanted_parts=1):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

In [11]:
list1, list2, list3, list4 = split_list(collectAccuracies,4)
frames = [list1, list2, list3, list4]
collectedAccuracies = pd.DataFrame(frames).T
collectedAccuracies.columns = ['2_groups_allROIs','3_groups_allROIs','9_groups_allROIs','13_groups_allROIs']


collectedAccuracies.to_csv('triangulate_allROIs.csv',sep=';',decimal='.', index=False)